In [21]:
base = base_Original

In [7]:
import pandas as pd
import numpy as np
import json
from openpyxl import Workbook, load_workbook

In [4]:
base_Original = pd.read_excel('task (3).xlsx')
base = pd.read_excel('task (3).xlsx')
base = base.sort_values(by='Flag Bolean')
#display(base)
media_base=(base['tempo estimado tarefa'].sum())/4
semanamin = base['Semana'].min()
semanamax = base['Semana'].max()
semana = semanamin
base_semana = base[(base.Semana == semana)]
soma_semana = base_semana['tempo estimado tarefa'].sum()

#variaveis HTML
asemana1 = semanamin
asemana2 = semana + 1
asemana3 = semana + 2
asemana4 = semana + 3

base_semana1 = base[(base.Semana == asemana1)]
base_semana2 = base[(base.Semana == asemana2)]
base_semana3 = base[(base.Semana == asemana3)]
base_semana4 = base[(base.Semana == asemana4)]

aTTS1 = round(base_semana1['tempo estimado tarefa'].sum(), 2)
aTTS2 = round(base_semana2['tempo estimado tarefa'].sum(), 2)
aTTS3 = round(base_semana3['tempo estimado tarefa'].sum(), 2)
aTTS4 = round(base_semana4['tempo estimado tarefa'].sum(), 2)

aQTD1 = len(base_semana1)
aQTD2 = len(base_semana2)
aQTD3 = len(base_semana3)
aQTD4 = len(base_semana4)



def medir_semana():
    base_semana_index = base[(base.Semana == semana)]
    soma_semana = base_semana_index['tempo estimado tarefa'].sum()
    if media_base+media_base*0.15 < soma_semana:
        resp = 'fora'
    else:
        resp = 'dentro'
    return resp


#faz nada essa função
def alterar_semana():
    base_semana = base[(base.Semana == semana)]
    firstbase_semana = base_semana['maint_order_id'][contador]
    base_semana.loc[base_semana['maint_order_id'] == firstbase_semana,'Semana'] = base_semana['Semana']+1  
    base_atualizada = base_semana
    return base_atualizada, firstbase_semana


#Equilibra semanas
def equilibrar_faze1(soma_semana,base_semana,semana):
    while media_base <= soma_semana:
        #print("Semana antes")
        #print(semana)
        #print(soma_semana)
        #print(media_base)
        firstbase_semana2 = base_semana['maint_order_id'].head(1).values
        firstbase_semana = int(firstbase_semana2)
        base.loc[base['maint_order_id'] == firstbase_semana,'Semana'] = base['Semana']+1 
        base_semana = base[(base.Semana == semana)]
        soma_semana = base_semana['tempo estimado tarefa'].sum()    
        #print("Semana depois")
        #print(semana)
        #print(firstbase_semana)
        #print(soma_semana)
        #print(media_base)
    return base, soma_semana, base_semana

def listar_models(i): 
    grupo3 = base['MODELO ATIVO']
    grupo3 = grupo3.tolist()
    grupo4 = np.unique(grupo3)
    output_listas = grupo4[i]
    return output_listas

#Semana menor
def semana_menor_HH(base,semanamin):
    base_semana = base[(base.Semana == semanamin)]
    soma_semana1 = base_semana['tempo estimado tarefa'].sum()
    semana1 = semanamin
    base_semana = base[(base.Semana == semanamin+1)]
    soma_semana2 = base_semana['tempo estimado tarefa'].sum()    
    semana2 = semanamin + 1
    base_semana = base[(base.Semana == semanamin+2)]
    soma_semana3 = base_semana['tempo estimado tarefa'].sum()
    semana3 = semanamin + 2
    base_semana = base[(base.Semana == semanamin+3)]
    soma_semana4 = base_semana['tempo estimado tarefa'].sum()
    semana4 = semanamin + 3
    data = np.array([[semana1, soma_semana1],[semana2, soma_semana2],[semana3, soma_semana3],[semana4, soma_semana4]])    
    data = pd.DataFrame(data, index=range(100,104),columns=['Semana', 'total'])
    menor_semana_HH = data.sort_values(by='total',ascending=False).head(1)
    return menor_semana_HH

def novas_colunas(base,semanamin):
    base.insert(loc=25, column='new_week', value = semanamin)
    base.insert(loc=26, column='distinct_sum', value=0)
    base.insert(loc=27, column='listagem_count', value=0)
    base.insert(loc=28, column='Grupo', value=0)
    base.insert(loc=28, column='concat_manut', value=0)
    
    return base

def colocar_coluna_distinct_sum(base,semanamin):
    base_teste = base
    count = base.groupby('Ativo+Tarefa+Ciclo')['BKNloja'].count()
    for item in base['Ativo+Tarefa+Ciclo']:
        resultado = count[item]
        base_teste.loc[base_teste['Ativo+Tarefa+Ciclo'] == item,'distinct_sum'] = resultado
    base = base_teste
    return base
def equalizar_planos_iguais(semanamin,semanamax,base):
    for i in range(semanamin,semanamax+1):
        base_semana = base[(base.new_week == semana)]
        print(i)
        for item in base_semana['id']:
            valor = max(base_semana.loc[base_semana['id'] == item,'distinct_sum'].values)
            if valor >=2:
                base_teste = base
                base_teste.rename(columns={'Ativo+Tarefa+Ciclo': 'cod_completo'}, inplace = True)
                var = base.loc[base['id'] == item,'cod_completo']
                var = var.tolist()
                var = var[0]
                base_teste = base_teste[(base_teste.cod_completo == var)]
                base_teste.sort_values(by='listagem_count')
                contador = 0
                for i in base_teste['id']:
                    base.loc[base['id'] == i,'listagem_count'] = contador
                    contador += 1
                CicloDias = max(base.loc[base['id'] == item,'CicloDias'].values)
                ciclosemanas = round(CicloDias/7,0)
                var_listagem_count = base.loc[base['id'] == item,'listagem_count']
                base.loc[base['id'] == item,'new_week'] = base['new_week']+(ciclosemanas*var_listagem_count)
    base = base.sort_values(by='cod_completo')
    return base
def criar_lista_critica(base):
    lista_ativos_criticos = base.groupby(by="concat_manut").sum()
    lista_ativos_criticos = lista_ativos_criticos.sort_values(by="Pont_total", ascending=True)
    lista_ativos_criticos = lista_ativos_criticos["Pont_total"]
    return lista_ativos_criticos

def alg_tratamento(base):
    semana = semanamin
    base_semana = base[(base.new_week == semana)]
    soma_semana = base_semana['tempo estimado tarefa'].sum()
    acao_max = len(criar_lista_critica(base))
    indice_ativos_criticos = 0
    ativo_menos_critico_fixo = criar_lista_critica(base).index[indice_ativos_criticos]
    ativo_menos_critico = criar_lista_critica(base).index[indice_ativos_criticos]
    total_lista = len(criar_lista_critica(base))

    for i in range(semanamin,semanamax+1):
        print( "---------------------------------------")
        print( "Semana"+ str(semana) )
        print( "---------------------------------------")

        base_semana = base[(base.new_week == semana)]
        soma_semana = base_semana['tempo estimado tarefa'].sum()    

        gb_Ativos =base_semana.groupby(by="CodigoAtivo").sum() 
        gb_Ativos_tempo= gb_Ativos['tempo estimado tarefa']


        for id2 in gb_Ativos_tempo.index:
            tmp = gb_Ativos['tempo estimado tarefa'][id2]

            tmpTotal = 0

            if tmp > media_base:
                base_semana2 = base_semana[(base_semana.CodigoAtivo == id2)]
                for id3 in base_semana2.index:

                    tmp3 = base_semana2['tempo estimado tarefa'][id3]
                    tmpTotal = tmpTotal + tmp3

                    if tmpTotal > tmp/2:
                        base_semana.loc[base_semana.index ==  id3,'Grupo'] = 2
                    else:
                        base_semana.loc[base_semana.index ==  id3,'Grupo'] = 1


        for x in base_semana.index: 
            texto = base_semana.Grupo[x]
            base.loc[base.index ==  x,'Grupo'] = texto               


        for x in base.index: 
            Grup = base.Grupo[x]
            CodAtv = base.CodigoAtivo[x]

            base.loc[base.index ==  x,'concat_manut'] = str(CodAtv) + "-grupo-" + str(Grup)


        #Parte 2

        base_semana = base[(base.new_week == semana)]   
        soma_semana = base_semana['tempo estimado tarefa'].sum()
        indice_ativos_criticos = 0
        print( "indice_ativos_criticos"+ str(indice_ativos_criticos) )




        lista_ativos_criticos= criar_lista_critica(base)

        for id in lista_ativos_criticos.index:



            atv_s = id
            print( "ativos"+ str(atv_s) )
            print(atv_s)

            if media_base*1.15 <= soma_semana:
                base.loc[base['concat_manut'] ==  atv_s,'new_week'] = base['new_week']+1


            base_semana = base[(base.new_week == semana)]   
            soma_semana = base_semana['tempo estimado tarefa'].sum() 


            print(soma_semana)
            print( "indice_ativos_criticos"+ str(indice_ativos_criticos) )


            if indice_ativos_criticos < total_lista-2:

                indice_ativos_criticos = indice_ativos_criticos + 1
                ativo_menos_critico = criar_lista_critica(base).index[indice_ativos_criticos]
                basesum=base.groupby(by="new_week").sum()
                basesum['tempo estimado tarefa']

        semana = semana+1
        indice_ativos_criticos = 0
        
    basesum=base.groupby(by="new_week").sum()
    basesum2=base.groupby(by="Semana").sum()    
    indice_semana_new = basesum['tempo estimado tarefa'].index.values
    resultados_semana_new = basesum['tempo estimado tarefa'].values
    indice_semana_old = basesum2['tempo estimado tarefa'].index.values
    resultados_semana_old = basesum2['tempo estimado tarefa'].values        
        
    S1 = 18
    base = base
        
    return base







#Programa

base.rename(columns={'Ativo+Tarefa+Ciclo': 'cod_completo'}, inplace = False)
novas_colunas(base,semanamin)
colocar_coluna_distinct_sum(base,semanamin)
equalizar_planos_iguais(semanamin,semanamax,base)
criar_lista_critica(base)
alg_tratamento(base)









base
#df.sort_values(by=['col1', 'col2'])

10
11
12
13
---------------------------------------
Semana10
---------------------------------------
indice_ativos_criticos0
ativosARC.19028-grupo-0
ARC.19028-grupo-0
12.783334862440825
indice_ativos_criticos0
ativosCAM.19028-grupo-0
CAM.19028-grupo-0
12.366667848080397
indice_ativos_criticos0
ativosBK084412-grupo-0
BK084412-grupo-0
11.950000550597906
indice_ativos_criticos0
ativosBK007301-grupo-1
BK007301-grupo-1
9.5333335660398
indice_ativos_criticos0
ativosBK062352-grupo-0
BK062352-grupo-0
7.783333282917738
indice_ativos_criticos0
ativosBK062355-grupo-0
BK062355-grupo-0
6.916666995733976
indice_ativos_criticos0
ativosBK007301-grupo-2
BK007301-grupo-2
4.250000428408384
indice_ativos_criticos0
ativosBK084407-grupo-0
BK084407-grupo-0
4.250000428408384
indice_ativos_criticos0
ativosBK084410-grupo-0
BK084410-grupo-0
4.250000428408384
indice_ativos_criticos0
ativosBK088406-grupo-0
BK088406-grupo-0
4.250000428408384
indice_ativos_criticos0
ativosBK055432-grupo-0
BK055432-grupo-0
4.25000042

,id,BKNloja,CodigoAtivo,Semana,maint_order_id,MODELO ATIVO,bs_Ord nome Ordem,tempo estimado tarefa,opened_at,scheduled_to,...,periodicity_hour,cod_completo,count,new_week,distinct_sum,listagem_count,concat_manut,Grupo,N_Crit,Pont_total
0,1,BKN19028,BK055432,10,419721,BROILER,PMP.BK.BRO.SPI.1002-BK - PREVENTIVA DE BROILER,0.500000,2022-02-20 10:50:25,2022-02-27 03:00:00,...,336,BK055432 - REALIZAR LIMPEZA NOS BICOS INJETORE...,3,10,3,0,BK055432-grupo-0,0,10,10
75,76,BKN19028,MRE.BKN19028,12,435703,MÁQUINA DE REFRIGERANTE,PMP.BK.MRE.SPI.1002-BK - AUDITORIA PREVENTIVAS...,0.016667,2022-03-13 10:42:20,2022-03-20 03:00:00,...,672,MRE.BKN19028 - VISA COOLER/FREEZER ESTÁ 100% O...,1,10,1,0,MRE.BKN19028-grupo-0,0,10,20
74,75,BKN19028,MRE.BKN19028,12,435703,MÁQUINA DE REFRIGERANTE,PMP.BK.MRE.SPI.1002-BK - AUDITORIA PREVENTIVAS...,0.016667,2022-03-13 10:42:20,2022-03-20 03:00:00,...,672,MRE.BKN19028 - ÚLTIMA PREVENTIVA/SANITIZAÇÃO O...,1,10,1,0,MRE.BKN19028-grupo-0,0,10,20
73,74,BKN19028,MRE.BKN19028,12,435703,MÁQUINA DE REFRIGERANTE,PMP.BK.MRE.SPI.1002-BK - AUDITORIA PREVENTIVAS...,0.016667,2022-03-13 10:42:20,2022-03-20 03:00:00,...,672,MRE.BKN19028 - TODOS OS SABORES ESTÃO COM AS E...,1,10,1,0,MRE.BKN19028-grupo-0,0,30,60
72,73,BKN19028,MRE.BKN19028,12,435703,MÁQUINA DE REFRIGERANTE,PMP.BK.MRE.SPI.1002-BK - AUDITORIA PREVENTIVAS...,0.016667,2022-03-13 10:42:20,2022-03-20 03:00:00,...,672,MRE.BKN19028 - TODOS OS COMPONENTES FILTRANTES...,1,10,1,0,MRE.BKN19028-grupo-0,0,30,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,31,BKN19028,BK007301,11,425349,MÁQUINA DE SORVETE,PMP.BK.MDS.SPI.C712.1002-BK - PREVENTIVA DE MÁ...,0.250000,2022-02-27 12:43:28,2022-03-06 03:00:00,...,672,BK007301 - Realizar Inspeção no conjunto da bo...,1,11,1,0,BK007301-grupo-2,2,10,10
29,30,BKN19028,BK007301,11,425349,MÁQUINA DE SORVETE,PMP.BK.MDS.SPI.C712.1002-BK - PREVENTIVA DE MÁ...,0.250000,2022-02-27 12:43:28,2022-03-06 03:00:00,...,672,BK007301 - Realizar Inspeção e limpeza nos plu...,1,11,1,0,BK007301-grupo-2,2,10,10
28,29,BKN19028,BK007301,11,425349,MÁQUINA DE SORVETE,PMP.BK.MDS.SPI.C712.1002-BK - PREVENTIVA DE MÁ...,0.500000,2022-02-27 12:43:28,2022-03-06 03:00:00,...,672,BK007301 - Realizar ajuste de viscosidade (4S),1,11,1,0,BK007301-grupo-2,2,10,10
27,28,BKN19028,BK007301,11,425349,MÁQUINA DE SORVETE,PMP.BK.MDS.SPI.C712.1002-BK - PREVENTIVA DE MÁ...,0.333333,2022-02-27 12:43:28,2022-03-06 03:00:00,...,672,BK007301 - Calibragem da temperatura do reserv...,1,11,1,0,BK007301-grupo-2,2,10,10


In [27]:
base = base_Original
base = base.sort_values(by='Flag Bolean')
#display(base)
media_base=(base['tempo estimado tarefa'].sum())/4
semanamin = base['Semana'].min()
semanamax = base['Semana'].max()
semana = semanamin
base_semana = base[(base.Semana == semana)]
soma_semana = base_semana['tempo estimado tarefa'].sum()

#variaveis HTML
asemana1 = semanamin
asemana2 = semana + 1
asemana3 = semana + 2
asemana4 = semana + 3

base_semana1 = base[(base.Semana == asemana1)]
base_semana2 = base[(base.Semana == asemana2)]
base_semana3 = base[(base.Semana == asemana3)]
base_semana4 = base[(base.Semana == asemana4)]

aTTS1 = round(base_semana1['tempo estimado tarefa'].sum(), 2)
aTTS2 = round(base_semana2['tempo estimado tarefa'].sum(), 2)
aTTS3 = round(base_semana3['tempo estimado tarefa'].sum(), 2)
aTTS4 = round(base_semana4['tempo estimado tarefa'].sum(), 2)

aQTD1 = len(base_semana1)
aQTD2 = len(base_semana2)
aQTD3 = len(base_semana3)
aQTD4 = len(base_semana4)


#base.rename(columns={'Ativo+Tarefa+Ciclo': 'cod_completo'}, inplace = False)
#novas_colunas(base,semanamin)
#colocar_coluna_distinct_sum(base,semanamin)
#equalizar_planos_iguais(semanamin,semanamax,base)
#criar_lista_critica(base)
#alg_tratamento(base)



#ajustando Base
base.rename(columns={'Ativo+Tarefa+Ciclo': 'cod_completo'}, inplace = False)


#novas_colunas(base,semanamin)
base.insert(loc=25, column='new_week', value = semanamin)
base.insert(loc=26, column='distinct_sum', value=0)
base.insert(loc=27, column='listagem_count', value=0)
base.insert(loc=28, column='Grupo', value=0)
base.insert(loc=28, column='concat_manut', value=0)

#colocar_coluna_distinct_sum(base,semanamin)
    
base_teste = base
count = base.groupby('Ativo+Tarefa+Ciclo')['BKNloja'].count()
for item in base['Ativo+Tarefa+Ciclo']:
	resultado = count[item]
	base_teste.loc[base_teste['Ativo+Tarefa+Ciclo'] == item,'distinct_sum'] = resultado
base = base_teste

#equalizar_planos_iguais(semanamin,semanamax,base)

for i in range(semanamin,semanamax+1):
	base_semana = base[(base.new_week == semana)]
	for item in base_semana['id']:
		valor = max(base_semana.loc[base_semana['id'] == item,'distinct_sum'].values)
		if valor >=2:
			base_teste = base
			base_teste.rename(columns={'Ativo+Tarefa+Ciclo': 'cod_completo'}, inplace = True)
			var = base.loc[base['id'] == item,'cod_completo']
			var = var.tolist()
			var = var[0]
			base_teste = base_teste[(base_teste.cod_completo == var)]
			base_teste.sort_values(by='listagem_count')
			contador = 0
			for i in base_teste['id']:
				base.loc[base['id'] == i,'listagem_count'] = contador
				contador += 1
			CicloDias = max(base.loc[base['id'] == item,'CicloDias'].values)
			ciclosemanas = round(CicloDias/7,0)
			var_listagem_count = base.loc[base['id'] == item,'listagem_count']
			base.loc[base['id'] == item,'new_week'] = base['new_week']+(ciclosemanas*var_listagem_count)
base = base.sort_values(by='cod_completo')
    

#criar_lista_critica(base)

lista_ativos_criticos = base.groupby(by="concat_manut").sum()
lista_ativos_criticos = lista_ativos_criticos.sort_values(by="Pont_total", ascending=True)
lista_ativos_criticos = lista_ativos_criticos["Pont_total"]


#alg_tratamento(base)

semana = semanamin
base_semana = base[(base.new_week == semana)]
soma_semana = base_semana['tempo estimado tarefa'].sum()
acao_max = len(criar_lista_critica(base))
indice_ativos_criticos = 0
ativo_menos_critico_fixo = criar_lista_critica(base).index[indice_ativos_criticos]
ativo_menos_critico = criar_lista_critica(base).index[indice_ativos_criticos]
total_lista = len(criar_lista_critica(base))

for i in range(semanamin,semanamax+1):
	base_semana = base[(base.new_week == semana)]
	soma_semana = base_semana['tempo estimado tarefa'].sum()    
	gb_Ativos =base_semana.groupby(by="CodigoAtivo").sum() 
	gb_Ativos_tempo= gb_Ativos['tempo estimado tarefa']
	for id2 in gb_Ativos_tempo.index:
		tmp = gb_Ativos['tempo estimado tarefa'][id2]
		tmpTotal = 0
		if tmp > media_base:
			base_semana2 = base_semana[(base_semana.CodigoAtivo == id2)]
			for id3 in base_semana2.index:
				tmp3 = base_semana2['tempo estimado tarefa'][id3]
				tmpTotal = tmpTotal + tmp3
				if tmpTotal > tmp/2:
					base_semana.loc[base_semana.index ==  id3,'Grupo'] = 2
				else:
					base_semana.loc[base_semana.index ==  id3,'Grupo'] = 1


	for x in base_semana.index: 
		texto = base_semana.Grupo[x]
		base.loc[base.index ==  x,'Grupo'] = texto               


	for x in base.index: 
		Grup = base.Grupo[x]
		CodAtv = base.CodigoAtivo[x]
		base.loc[base.index ==  x,'concat_manut'] = str(CodAtv) + "-grupo-" + str(Grup)


        #Parte 2

	base_semana = base[(base.new_week == semana)]   
	soma_semana = base_semana['tempo estimado tarefa'].sum()
	indice_ativos_criticos = 0
	lista_ativos_criticos= criar_lista_critica(base)
	for id in lista_ativos_criticos.index:
		atv_s = id
		if media_base*1.15 <= soma_semana:
			base.loc[base['concat_manut'] ==  atv_s,'new_week'] = base['new_week']+1
		base_semana = base[(base.new_week == semana)]   
		soma_semana = base_semana['tempo estimado tarefa'].sum() 
		if indice_ativos_criticos < total_lista-2:
			indice_ativos_criticos = indice_ativos_criticos + 1
			ativo_menos_critico = criar_lista_critica(base).index[indice_ativos_criticos]
			basesum=base.groupby(by="new_week").sum()
			basesum['tempo estimado tarefa']
	semana = semana+1
	indice_ativos_criticos = 0
        
basesum=base.groupby(by="new_week").sum()
basesum2=base.groupby(by="Semana").sum()    
indice_semana_new = basesum['tempo estimado tarefa'].index.values.tolist()
resultados_semana_new = basesum['tempo estimado tarefa'].values.tolist()
indice_semana_old = basesum2['tempo estimado tarefa'].index.values.tolist()
resultados_semana_old = basesum2['tempo estimado tarefa'].values.tolist()        
      
S1 = 18
base = base
        



resultados_semana_old

json_resultados_semana_old = json.dumps(resultados_semana_old)
json_resultados_semana_old 


'[4.4500012546777725, 3.9166669696569443, 6.533334132283926, 3.250001087784767]'

In [25]:
basesum=base.groupby(by="Semana").sum()
basesum['tempo estimado tarefa']

Semana
10    4.450001
11    3.916667
12    6.533334
13    3.250001
Name: tempo estimado tarefa, dtype: float64

In [26]:
basesum=base.groupby(by="new_week").sum()
basesum['Pont_total']

new_week
10    1220
11     290
12     240
13     170
14     330
17      60
21      20
Name: Pont_total, dtype: int64

In [27]:
basesum=base.groupby(by="Semana").sum()
basesum['Pont_total']


Semana
10    640
11    150
12    900
13    640
Name: Pont_total, dtype: int64

In [13]:
base.to_excel("tratado2002.xlsx")


In [19]:
myarrey = [1,2,3,4]

teste = json.dumps(myarrey)


In [17]:
import json

In [25]:
tt = np.array(resultados_semana_new)
tt = tt.tolist()
tt

[4.250000428408384,
 4.366666853427887,
 5.000000447034836,
 1.250000610947609,
 2.0333337783813477,
 0.41666729748249054,
 0.41666701436042786,
 0.41666701436042786]

In [28]:
total = len(resultados_semana_old)

In [29]:
total.tolist()

4

In [37]:
valor = media_base*1.15
quantidade = len(resultados_semana_old)
meu_array = [valor] * quantidade,
meu_array = list(meu_array)
meu_array = meu_array
meu_array



[[5.21812599026598, 5.21812599026598, 5.21812599026598, 5.21812599026598]]

In [38]:
minha_tupla = (1, 2, 3, 4, 5)
minha_lista = list(minha_tupla)
print(minha_lista)

[1, 2, 3, 4, 5]


In [40]:
valor = media_base*1.15
quantidade = len(resultados_semana_old)
limite1 = [valor] * quantidade
limite1 = list(limite1)
json_limite1 = json.dumps(limite1)


In [56]:
    basesum = base.groupby(by="new_week").sum()
    basesum2 = base.groupby(by="Semana").sum()
    indice_semana_new = basesum['tempo estimado tarefa'].index.values.tolist()
    resultados_semana_new = basesum['tempo estimado tarefa'].values.tolist()
    indice_semana_old = basesum2['tempo estimado tarefa'].index.values.tolist()
    resultados_semana_old = basesum2['tempo estimado tarefa'].values.tolist()

    json_resultados_semana_old = json.dumps(resultados_semana_old)
    json_indice_semana_old = json.dumps(indice_semana_old)

    json_resultados_semana_new = json.dumps(resultados_semana_new)
    json_indice_semana_new = json.dumps(indice_semana_new)   

In [42]:
json_indice_semana_new

'[10, 11, 12, 13, 14, 16, 17, 21]'

In [43]:
json_resultados_semana_new

'[4.250000428408384, 4.366666853427887, 5.000000447034836, 1.250000610947609, 2.0333337783813477, 0.41666729748249054, 0.41666701436042786, 0.41666701436042786]'

In [113]:
grouped = base.groupby(['CodigoAtivo', 'Semana'])['tempo estimado tarefa'].sum()
pivot_table = grouped.reset_index().pivot(index='CodigoAtivo', columns='Semana', values='tempo estimado tarefa')
pivot_table.fillna(0, inplace=True)   
data_old = pivot_table.round(2)
data_old


Semana,10,11,12,13
CodigoAtivo,,,,
ARC.19028,0.00,0.00,0.83,0.00
BK007301,0.00,3.92,1.17,0.00
BK055432,2.25,0.00,1.08,1.67
BK062352,0.17,0.00,1.75,0.00
BK062355,0.00,0.00,0.87,0.00
BK084407,0.50,0.00,0.08,0.20
BK084410,0.20,0.00,0.00,0.35
BK084412,0.42,0.00,0.00,0.42
BK088406,0.50,0.00,0.08,0.20


In [ ]:
data_old

    

In [112]:
data_old

Semana,10,11,12,13
CodigoAtivo,,,,
ARC.19028,0.00,0.00,0.83,0.00
BK007301,0.00,3.92,1.17,0.00
BK055432,2.25,0.00,1.08,1.67
BK062352,0.17,0.00,1.75,0.00
BK062355,0.00,0.00,0.87,0.00
BK084407,0.50,0.00,0.08,0.20
BK084410,0.20,0.00,0.00,0.35
BK084412,0.42,0.00,0.00,0.42
BK088406,0.50,0.00,0.08,0.20


In [82]:
grouped = base.groupby(['CodigoAtivo', 'Semana'])['tempo estimado tarefa'].sum()
pivot_table = grouped.reset_index().pivot(index='CodigoAtivo', columns='Semana', values='tempo estimado tarefa')
pivot_table.fillna(0, inplace=True)   
data_old = pivot_table.round(2)
data_old

Semana,10,11,12,13
CodigoAtivo,,,,
ARC.19028,0.00,0.00,0.83,0.00
BK007301,0.00,3.92,1.17,0.00
BK055432,2.25,0.00,1.08,1.67
BK062352,0.17,0.00,1.75,0.00
BK062355,0.00,0.00,0.87,0.00
BK084407,0.50,0.00,0.08,0.20
BK084410,0.20,0.00,0.00,0.35
BK084412,0.42,0.00,0.00,0.42
BK088406,0.50,0.00,0.08,0.20


In [73]:
import pandas as pd

# Cria um dataframe
df = pd.DataFrame({'A': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'],
                   'B': ['one', 'one', 'two', 'two', 'two', 'one'],
                   'C': [1, 2, 3, 4, 5, 6],
                   'D': [10, 20, 30, 40, 50, 60]})

# Realiza o groupby em duas colunas e aplica a soma em uma terceira coluna
grouped = df.groupby(['A', 'B'])['D'].sum()

# Aplica o método pivot() para transformar as linhas do groupby em colunas
pivot_table = grouped.reset_index().pivot(index='A', columns='B', values='D')

# Imprime o resultado
print(pivot_table)



B    one  two
A            
bar   60   90
foo   30   30


In [114]:
df = pd.DataFrame({'A': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'],
                   'B': ['one', 'one', 'two', 'two', 'two', 'one'],
                   'C': [1, 2, 3, 4, 5, 6],
                   'D': [10, 20, 30, 40, 50, 60]})

In [115]:
df

,A,B,C,D
0,foo,one,1,10
1,foo,one,2,20
2,foo,two,3,30
3,bar,two,4,40
4,bar,two,5,50
5,bar,one,6,60


In [116]:
# Carrega os dados em um dataframe
df = pd.read_csv("exemplo.csv")

# Converte o dataframe em uma lista de dicionários
data = df.to_dict("records")

# Renderiza o template HTML com os dados
data

[{'company_name': nan,
  'is_closed': 'ABERTO',
  'store_bk_number': 18127,
  'region_name': 'ADISER SP1',
  'sector_name': 'SP',
  'main_werks_id': 10.0,
  'store_bkn_name': '18127 - SHOPPING MAIS LARGO 13'},
 {'company_name': nan,
  'is_closed': 'ABERTO',
  'store_bk_number': 18368,
  'region_name': 'ADISER SP1',
  'sector_name': 'SP',
  'main_werks_id': 15.0,
  'store_bkn_name': '18368 - SHOPPING MOOCA PLAZA'},
 {'company_name': nan,
  'is_closed': 'ABERTO',
  'store_bk_number': 18554,
  'region_name': 'ADISER SP1',
  'sector_name': 'SP',
  'main_werks_id': 16.0,
  'store_bkn_name': '18554 - SHOPPING JK IGUATEMI'},
 {'company_name': nan,
  'is_closed': 'ABERTO',
  'store_bk_number': 18343,
  'region_name': 'ADISER SP1',
  'sector_name': 'SP',
  'main_werks_id': 18.0,
  'store_bkn_name': '18343 - SHOPPING THE SQUARE GRANJA VIANNA'},
 {'company_name': nan,
  'is_closed': 'ABERTO',
  'store_bk_number': 18548,
  'region_name': 'ADISER SC',
  'sector_name': 'SUL',
  'main_werks_id': 19.0